## Spin up example
This notebook is an example of how to use the tP4 to automate model spin-up by rerunning the same forcings until a desired groundwater level is obtained. (NEEDS TO BE CLEANED UP W/ NOTES FOR CLARITY). ALSO: Currently this assumes model has been run once--look at spinup.py for better example on how to execute spinup.

In [ ]:
import sys
from datetime import datetime
import os
os.chdir("../..")


extra_path = "/Users/lraming/Documents/Repos/GP4"
if extra_path not in sys.path:
    sys.path.append(extra_path)

from tribsmodel import Model as model

m = model()

In [ ]:
input_file = 'sycMS2.in'

base_name = 'ms2_'

m.read_input_file(input_file)

# gw voi file from intial run will want to update in future
m.options["optgwfile"]["value"] = 1
gwvoi = "results/2023-10-19/ms2_0_gwvoi"
end_time = int(m.options["runtime"]["value"])


In [ ]:
m.check_paths()

In [ ]:
### Currently this assumes model has been run once--look at spinup.py for better example on how to execute spinup

# gw voi file from intial run will want to update in future
m.options["optgwfile"]["value"] = 1
gwvoi = "results/2023-10-19/ms2_0_gwvoi"
end_time = int(m.options["runtime"]["value"])

# Get today's date and make results directory
today = datetime.now().date()
today = today.strftime('%Y-%m-%d')
todays_results = 'results/' + today + '/'

try:
    os.makedirs(todays_results)
except FileExistsError:
    # directory already exists
    pass


# currently just running spinup for 3 years total, but maybe better approach.

iterations = [1, 2]

for i in iterations:
    result_path = todays_results + str(i)
    
    try:
        os.makedirs(result_path)
    except FileExistsError:
        # directory already exists
        pass
  
    outfiles = result_path + '/' + base_name
    m.options["outfilename"]["value"] = outfiles
    m.options["outhydrofilename"]["value"] = outfiles
    m.options['gwaterfile']['value'] = gwvoi
    m.write_input_file(input_file)
    m.run('./tRIBS', input_file, mpi_command='mpirun -n 4')

    # update gwvoi file
    gwvoi = m.options["outfilename"]["value"] + str(i) + "_gwvoi"

    # merger parallel files and write ouyt gwvoi file
    df = m.merge_parllel_spatial_files(dtime=end_time, write=False)
    df = df[str(end_time)]
    df[['ID', 'Nwt']].to_csv(gwvoi, sep='\t', index=False, header=False)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# GW runs to look at: gwvoi in 1, gwvoi in 2, and 8700.d in 2
column_names=["ID","Nwt"]
runs = [df, pd.read_csv("results/2023-10-23/1/ms2_1_gwvoi",names=column_names,sep='\t'), pd.read_csv("results/2023-10-19/ms2_0_gwvoi",names=column_names,sep='\t')] 
titles = [0,1,2]

In [ ]:
for run,title in zip(runs,titles):
    ax = run["Nwt"].plot(kind='hist', alpha=0.5, legend=True, label=title)
ax.set_xlabel('Nwt (mm)')

__Mean difference between runs 1 and 2__

In [ ]:
# mean difference between runs 1 and 2
abs(np.mean(df.Nwt-runs[1]['Nwt']))

__Mean difference between runs 0 and 2__

In [ ]:
# mean difference between runs 0 and 1
abs(np.mean(runs[2]['Nwt']-runs[0]['Nwt']))